# LAB 1 - Python & Pandas Foundations (Oil & Gas Context)
**Course:** Data Science with Python  
**Lab:** 1 of 4  
**Last updated:** 2025-12-26

## Goal
Learn the Python/Pandas workflow for loading, inspecting, cleaning, and transforming tabular data.

## Dataset options (choose ONE)
1) Kaggle search (download CSV locally): https://www.kaggle.com/datasets (search: *oil production*, *oil and gas production*)  
2) Offline-friendly synthetic dataset generator (included)

> This notebook supports **local CSV** and a **synthetic fallback** to work in restricted training environments.

## 1) Setup

In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

print("pandas:", pd.__version__)

## 2) Load data (Local CSV or Synthetic Fallback)

In [ ]:
from pathlib import Path

DATA_PATH = Path("data/oil_production_sample.csv")  # <-- change to your CSV location

def make_synthetic_oil_production(n_rows=365, seed=42):
    rng = np.random.default_rng(seed)
    dates = pd.date_range("2024-01-01", periods=n_rows, freq="D")
    field = rng.choice(["NorthSea-A", "NorthSea-B", "GOM-Alpha", "ME-East"], size=n_rows, p=[0.30,0.25,0.25,0.20])
    oil_bbl = rng.normal(52000, 8000, size=n_rows).clip(15000, 90000).round(0)
    gas_mcf = (oil_bbl * rng.normal(3.8, 0.6, size=n_rows)).clip(10000, 500000).round(0)
    water_cut = rng.normal(0.28, 0.08, size=n_rows).clip(0.05, 0.65)
    downtime_hr = rng.poisson(1.2, size=n_rows).astype(float)
    # inject some missingness and bad values
    mask = rng.random(n_rows) < 0.04
    water_cut[mask] = np.nan
    oil_bbl[rng.random(n_rows) < 0.01] = -999  # placeholder/bad reading
    downtime_hr[rng.random(n_rows) < 0.03] = np.nan

    df = pd.DataFrame({
        "date": dates,
        "field": field,
        "oil_bbl": oil_bbl,
        "gas_mcf": gas_mcf,
        "water_cut": water_cut,
        "downtime_hr": downtime_hr,
    })
    return df

if DATA_PATH.exists():
    df = pd.read_csv(DATA_PATH)
    print("Loaded:", DATA_PATH, "shape:", df.shape)
else:
    df = make_synthetic_oil_production()
    print("Using synthetic dataset. shape:", df.shape)

df.head()

## 3) Quick inspection

In [ ]:
df.info()

In [ ]:
df.describe(include="all").T

## 4) Cleaning tasks

### 4.1 Parse dates and fix types

In [ ]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")
num_cols = ["oil_bbl", "gas_mcf", "water_cut", "downtime_hr"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df.dtypes

### 4.2 Replace placeholder values (e.g. -999)

In [ ]:
df["oil_bbl"] = df["oil_bbl"].replace([-999, -999.0], np.nan)
df.isna().mean().sort_values(ascending=False)

### 4.3 Handle missing values

In [ ]:
df["downtime_hr"] = df["downtime_hr"].fillna(0)
df["water_cut"] = df.groupby("field")["water_cut"].transform(lambda s: s.fillna(s.median()))
df.isna().sum()

## 5) Feature engineering

In [ ]:
df["oil_bbl_per_hr_down"] = df["oil_bbl"] / df["downtime_hr"].replace(0, np.nan)
df["gor_mcf_per_bbl"] = df["gas_mcf"] / df["oil_bbl"]
df.head()

## 6) Filtering, sorting

In [ ]:
northsea = df[df["field"].str.contains("NorthSea")].sort_values("date")
northsea.head()

## 7) Groupby and aggregation

In [ ]:
daily_field = df.groupby(["date","field"], as_index=False).agg(
    oil_bbl=("oil_bbl","sum"),
    gas_mcf=("gas_mcf","sum"),
    avg_water_cut=("water_cut","mean"),
    downtime_hr=("downtime_hr","sum"),
)
daily_field.head()

## 8) Export cleaned data (for Lab 2)

In [ ]:
from pathlib import Path
OUT_PATH = Path("outputs/lab1_cleaned_oil_production.csv")
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
daily_field.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)

## Checkpoint questions
1) What does `df.info()` tell you that `df.describe()` does not?  
2) Why can replacing placeholder codes with NaN be important before analysis?  
3) When would `groupby().transform()` be preferred over `groupby().agg()`?